In [ ]:
# Transformers installation
! pip install transformers datasets


# Summary of the tasks

На этой странице представлены наиболее распространенные варианты использования библиотеки. Доступные модели позволяют создавать множество различных
конфигураций и отличаются большой универсальностью в использовании. Здесь представлены наиболее простые из них, демонстрирующие использование для
таких задач, как классификация изображений, ответы на вопросы, классификация последовательностей, распознавание именованных объектов и другие.

В этих примерах используются автомодели, которые представляют собой классы, создающие экземпляр модели в соответствии с заданной контрольной
точкой, автоматически выбирая правильную архитектуру модели. Пожалуйста, ознакомьтесь с [AutoModel](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModel) документация
для получения дополнительной информации. Не стесняйтесь изменять код, чтобы он был более конкретным, и адаптировать его к вашему конкретному варианту использования.

Чтобы модель хорошо справлялась с задачей, она должна быть загружена из контрольной точки, соответствующей этой задаче. Эти контрольные точки обычно предварительно обрабатываются на большом массиве данных и настраиваются для конкретной задачи.
 Это означает
следующее:

- Не все модели были точно настроены для всех задач. Если вы хотите точно настроить модель для конкретной задачи, вы можете использовать
один из следующих способов *run_$TASK.py* scripts in the [examples](https://github.com/huggingface/transformers/tree/main/examples) directory.
- Доработанные модели были доработаны на основе определенного набора данных. Этот набор данных может совпадать, а может и не совпадать с вашим вариантом использования и
доменом. Как упоминалось ранее, вы можете использовать [examples](https://github.com/huggingface/transformers/tree/main/examples) scripts для точной настройки вашей модели или вы можете
создать свой собственный обучающий скрипт.

Для выполнения логического вывода по задаче в библиотеке доступно несколько механизмов:

- Конвейеры: очень простые в использовании абстракции, для которых требуется всего две строки кода.
- Прямое использование модели: меньше абстракций, но больше гибкости и мощности благодаря прямому доступу к токенизатору
  (PyTorch/TensorFlow) и полная способность к логическому выводу.

Здесь представлены оба подхода.

<Tip>

Все представленные здесь задачи используют предварительно подготовленные контрольные точки, которые были точно настроены для конкретных задач. Загрузка
контрольной точки, которая не была точно настроена для конкретной задачи, приведет к загрузке только базовых уровней трансформатора, а не
дополнительной головки, которая используется для задачи, что приведет к случайной инициализации весов этой головки.

Это привело бы к случайному выводу данных.

</Tip>

## Sequence Classification

Классификация последовательностей - это задача классификации последовательностей в соответствии с заданным числом классов. Примером классификации
последовательностей является набор данных GLUE, который полностью основан на этой задаче. Если вы хотите точно настроить
модель для задачи классификации последовательностей GLUE, вы можете использовать [run_glue.py](https://github.com/huggingface/transformers/tree/main/examples/pytorch/text-classification/run_glue.py), [run_tf_glue.py](https://github.com/huggingface/transformers/tree/main/examples/tensorflow/text-classification/run_tf_glue.py), [run_tf_text_classification.py](https://github.com/huggingface/transformers/tree/main/examples/tensorflow/text-classification/run_tf_text_classification.py) or [run_xnli.py](https://github.com/huggingface/transformers/tree/main/examples/pytorch/text-classification/run_xnli.py) scripts.

Вот пример использования конвейеров для анализа настроений: определение того, является ли последовательность положительной или отрицательной. В
sst2 используется точно настроенная модель, которая является задачей СКЛЕИВАНИЯ.

Это возвращает метку ("POSITIVE" or "NEGATIVE") наряду со счетом, следующим образом:

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

result = classifier("I hate you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: NEGATIVE, with score: 0.9991

In [ ]:
result = classifier("I love you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9999

Вот пример выполнения классификации последовательностей с использованием модели, чтобы определить, являются ли две последовательности парафразами
друг друга. Процесс заключается в следующем:

1. Создайте экземпляр токенизатора и модели из имени контрольной точки. Модель идентифицируется как модель BERT и загружается
с весами, сохраненными в контрольной точке.
2. Создайте последовательность из двух предложений с правильными разделителями для конкретной модели, идентификаторами типов токенов и
масками внимания (которые будут автоматически созданы токенизатором).
3. Пропустите эту последовательность через модель так, чтобы она была отнесена к одному из двух доступных классов: 0 (не является
перефразировкой) и 1 (является перефразировкой).
4. Вычислите softmax результата, чтобы получить вероятности для всех классов.
5. Распечатайте результаты.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to
# the sequence, as well as compute the attention masks.
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase).logits
not_paraphrase_classification_logits = model(**not_paraphrase).logits

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

not paraphrase: 10%
is paraphrase: 90%

In [ ]:
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

not paraphrase: 94%
is paraphrase: 6%

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to
# the sequence, as well as compute the attention masks.
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="tf")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="tf")

paraphrase_classification_logits = model(paraphrase).logits
not_paraphrase_classification_logits = model(not_paraphrase).logits

paraphrase_results = tf.nn.softmax(paraphrase_classification_logits, axis=1).numpy()[0]
not_paraphrase_results = tf.nn.softmax(not_paraphrase_classification_logits, axis=1).numpy()[0]

# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

not paraphrase: 10%
is paraphrase: 90%

In [ ]:
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

not paraphrase: 94%
is paraphrase: 6%

## Extractive Question Answering

Экстрактивный ответ на вопрос - это задача извлечения ответа из текста, заданного вопросом. Примером
набора данных для ответа на вопрос является набор данных SQuAD, который полностью основан на этой задаче. Если вы хотите точно настроить
модель для выполнения задачи отряда, вы можете использовать [run_qa.py](https://github.com/huggingface/transformers/tree/main/examples/pytorch/question-answering/run_qa.py) and
[run_tf_squad.py](https://github.com/huggingface/transformers/tree/main/examples/tensorflow/question-answering/run_tf_squad.py)
scripts.


Вот пример использования конвейеров для ответа на вопрос: извлечение ответа из текста, заданного вопросом. В нем
используется тонко настроенная модель для SQuAD.

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
"""

Это возвращает ответ, извлеченный из текста, оценку достоверности, а также значения "начало" и "конец", которые являются
позициями извлеченного ответа в тексте.

In [ ]:
result = question_answerer(question="What is extractive question answering?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)

Answer: 'the task of extracting an answer from a text given a question', score: 0.6177, start: 34, end: 95

In [ ]:
result = question_answerer(question="What is a good example of a question answering dataset?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)

Answer: 'SQuAD dataset', score: 0.5152, start: 147, end: 160

Вот пример ответа на вопрос с использованием модели и токенизатора. Процесс заключается в следующем:

1. Создайте токенизатор и модель из имени контрольной точки. Модель идентифицируется как модель BERT и загружается
с весами, сохраненными в контрольной точке.
2. Сформулируйте текст и несколько вопросов.
3. Повторите вопросы и создайте последовательность из текста и текущего вопроса, используя соответствующие
модели разделители, идентификаторы типов токенов и маски внимания.
4. Пропустите эту последовательность через модель. Это выводит диапазон оценок по всем жетонам последовательности (вопрос и
текст), как для начальной, так и для конечной позиций.
5. Вычислите softmax результата, чтобы получить вероятности для всех жетонов.
6. Извлеките токены из определенных значений start и stop, преобразуйте эти токены в строку.
7. Распечатайте результаты.

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    # Get the most likely end of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )

    print(f"Question: {question}")
    print(f"Answer: {answer}")

Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="tf")
    input_ids = inputs["input_ids"].numpy()[0]

    outputs = model(inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]
    # Get the most likely end of answer with the argmax of the score
    answer_end = tf.argmax(answer_end_scores, axis=1).numpy()[0] + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )

    print(f"Question: {question}")
    print(f"Answer: {answer}")

Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch

## Language Modeling

Языковое моделирование - это задача подгонки модели к корпусу данных, который может быть специфичен для конкретной предметной области. Все популярные
модели на основе transformer обучаются с использованием различных вариантов языкового моделирования, например, BERT с использованием маскированного языкового моделирования,
GPT-2 с использованием каузального языкового моделирования.

Языковое моделирование может быть полезно и вне предварительной подготовки, например, для изменения распределения моделей
в зависимости от предметной области: использование языковой модели, обученной на очень большом корпусе, а затем ее точная настройка для набора данных новостей или
научных статей, например. [LysandreJik/arxiv-nlp](https://huggingface.co/lysandre/arxiv-nlp).

### Masked Language Modeling

Моделирование маскируемого языка - это задача маскирования токенов в последовательности с помощью маскирующего токена и запроса модели
заполнить эту маску соответствующим токеном. Это позволяет модели учитывать как правый контекст (токены
справа от маски), так и левый контекст (токены слева от маски). Такое обучение создает прочную основу для
последующих задач, требующих двунаправленного контекста, таких как SQuAD (question answering, see [Lewis, Lui, Goyal et al.](https://arxiv.org/abs/1910.13461), part 4.2). If you would like to fine-tune a model on a masked language modeling
task, you may leverage the [run_mlm.py](https://github.com/huggingface/transformers/tree/main/examples/pytorch/language-modeling/run_mlm.py) script.

Вот пример использования конвейеров для замены маски из последовательности:

In [ ]:
from transformers import pipeline

unmasker = pipeline("fill-mask")

Это выводит последовательности с заполненной маской, оценкой достоверности и идентификатором токена в словаре токенизатора:

In [ ]:
from pprint import pprint

pprint(
    unmasker(
        f"HuggingFace is creating a {unmasker.tokenizer.mask_token} that the community uses to solve NLP tasks."
    )
)

[{'score': 0.1793,
  'sequence': 'HuggingFace is creating a tool that the community uses to solve '
              'NLP tasks.',
  'token': 3944,
  'token_str': ' tool'},
 {'score': 0.1135,
  'sequence': 'HuggingFace is creating a framework that the community uses to '
              'solve NLP tasks.',
  'token': 7208,
  'token_str': ' framework'},
 {'score': 0.0524,
  'sequence': 'HuggingFace is creating a library that the community uses to '
              'solve NLP tasks.',
  'token': 5560,
  'token_str': ' library'},
 {'score': 0.0349,
  'sequence': 'HuggingFace is creating a database that the community uses to '
              'solve NLP tasks.',
  'token': 8503,
  'token_str': ' database'},
 {'score': 0.0286,
  'sequence': 'HuggingFace is creating a prototype that the community uses to '
              'solve NLP tasks.',
  'token': 17715,
  'token_str': ' prototype'}]

Ниже приведен пример моделирования замаскированного языка с использованием модели и токенизатора. Процесс заключается в следующем:

1. Создайте экземпляр токенизатора и модели из имени контрольной точки. Модель идентифицируется как модель Дистильберта и
загружается с весами, сохраненными в контрольной точке.
2. Определите последовательность с замаскированным токеном, поместив `tokenizer.mask_token` вместо слова.
3. Закодируйте эту последовательность в список идентификаторов и найдите позицию замаскированного токена в этом списке.
4. Извлеките прогнозы по индексу токена маски: этот тензор имеет тот же размер, что и словарь, а
значения представляют собой оценки, присвоенные каждому токену. Модель присваивает более высокие оценки токенам, которые она считает вероятными в данном
контексте.
5. Извлеките 5 лучших токенов, используя методы PyTorch `topk` или TensorFlow `top_k`.
6. Замените токен маски на токены и распечатайте результаты

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

sequence = (
    "Distilled models are smaller than the models they mimic. Using them instead of the large "
    f"versions would help {tokenizer.mask_token} our carbon footprint."
)

inputs = tokenizer(sequence, return_tensors="pt")
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help offset our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help improve our carbon footprint.

In [ ]:
from transformers import TFAutoModelForMaskedLM, AutoTokenizer
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = TFAutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

sequence = (
    "Distilled models are smaller than the models they mimic. Using them instead of the large "
    f"versions would help {tokenizer.mask_token} our carbon footprint."
)

inputs = tokenizer(sequence, return_tensors="tf")
mask_token_index = tf.where(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]

token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = tf.math.top_k(mask_token_logits, 5).indices.numpy()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help offset our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help improve our carbon footprint.

При этом выводятся пять последовательностей с 5 лучшими токенами, предсказанными моделью.

### Causal Language Modeling

Каузальное языковое моделирование - это задача прогнозирования токена, следующего за последовательностью токенов. В этой ситуации
модель учитывает только левый контекст (токены слева от маски). Такое обучение особенно интересно
для задач генерации. Если вы хотите точно настроить модель в задаче моделирования каузального языка, вы можете использовать
[run_clm.py](https://github.com/huggingface/transformers/tree/main/examples/pytorch/language-modeling/run_clm.py) script.

Обычно следующий токен предсказывается путем выборки логитов последнего скрытого состояния, которое модель выдает из
входной последовательности.

Вот пример использования токенизатора и модели и использования
[top_k_top_p_filtering()](https://huggingface.co/docs/transformers/main/en/internal/generation_utils#transformers.top_k_top_p_filtering) метод для выборки следующего токена, следующего за входной последовательностью
токенов.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

sequence = f"Hugging Face is based in DUMBO, New York City, and"

inputs = tokenizer(sequence, return_tensors="pt")
input_ids = inputs["input_ids"]

# get logits of last hidden state
next_token_logits = model(**inputs).logits[:, -1, :]

# filter
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

# sample
probs = nn.functional.softmax(filtered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)

generated = torch.cat([input_ids, next_token], dim=-1)

resulting_string = tokenizer.decode(generated.tolist()[0])
print(resulting_string)

Hugging Face is based in DUMBO, New York City, and ...

Вот пример использования токенизатора и модели, а также использования
[tf_top_k_top_p_filtering()](https://huggingface.co/docs/transformers/main/en/internal/generation_utils#transformers.tf_top_k_top_p_filtering) метод для выборки следующего токена, следующего за входной последовательностью
токенов.

In [ ]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer, tf_top_k_top_p_filtering
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = TFAutoModelForCausalLM.from_pretrained("gpt2")

sequence = f"Hugging Face is based in DUMBO, New York City, and"

inputs = tokenizer(sequence, return_tensors="tf")
input_ids = inputs["input_ids"]

# get logits of last hidden state
next_token_logits = model(**inputs).logits[:, -1, :]

# filter
filtered_next_token_logits = tf_top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

# sample
next_token = tf.random.categorical(filtered_next_token_logits, dtype=tf.int32, num_samples=1)

generated = tf.concat([input_ids, next_token], axis=1)

resulting_string = tokenizer.decode(generated.numpy().tolist()[0])
print(resulting_string)

Hugging Face is based in DUMBO, New York City, and ...

Это выводит (надеюсь) последовательный следующий токен, следующий за исходной последовательностью, которой в нашем случае является слово *is* or
*features*.

В следующем разделе мы покажем, как [generation.GenerationMixin.generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) может использоваться для
генерации нескольких токенов заданной длины вместо одного токена за раз.

### Text Generation

генерация текста (также известная как генерация открытого текста) цель состоит в том, чтобы создать связную часть текста, которая является
продолжением заданного контекста. В следующем примере показано, как *GPT-2* можно использовать в конвейерах для генерации текста.
По умолчанию во всех моделях применяется функция отбора проб *Top-K* при использовании в трубопроводах, как указано в соответствующих конфигурациях
(see [gpt-2 config](https://huggingface.co/gpt2/blob/main/config.json) for example).

In [ ]:
from transformers import pipeline

text_generator = pipeline("text-generation")
print(text_generator("As far as I am concerned, I will", max_length=50, do_sample=False))

[{'generated_text': 'As far as I am concerned, I will be the first to admit that I am not a fan of the idea of a
"free market." I think that the idea of a free market is a bit of a stretch. I think that the idea'}]

Здесь модель генерирует случайный текст общей максимальной длиной *50* токенов из контекста * "Насколько я
понимаю, я буду"*. За кулисами объект конвейера вызывает метод
[PreTrainedModel.generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) для генерации текста. Аргументы по умолчанию для этого метода могут быть
переопределены в конвейере, как показано выше для аргументов `max_length` и `do_sample`.

Ниже приведен пример генерации текста с использованием "XLNet" и его токенизатора, который включает в себя прямой вызов `generate()`:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("xlnet-base-cased")
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""

prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

prompt_length = len(tokenizer.decode(inputs[0]))
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length + 1 :]

print(generated)

Today the weather is really nice and I am planning ...

In [ ]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer

model = TFAutoModelForCausalLM.from_pretrained("xlnet-base-cased")
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""

prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="tf")["input_ids"]

prompt_length = len(tokenizer.decode(inputs[0]))
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length + 1 :]

print(generated)

Today the weather is really nice and I am planning ...

В настоящее время генерация текста возможна с помощью *GPT-2*, *OpenAI-GPT*, *CTRL*, *XLNet*, *Transfo-XL* и *Reformer* в
PyTorch, а также для большинства моделей в Tensorflow. Как видно из приведенного выше примера, *XLNet* и *Transfo-XL* часто
нуждаются в дополнении, чтобы хорошо работать. GPT-2, как правило, является хорошим выбором для генерации открытого текста, поскольку он был разработан
на миллионах веб-страниц с целью моделирования причинно-следственных связей.

Для получения дополнительной информации о том, как применять различные стратегии декодирования для генерации текста, пожалуйста, ознакомьтесь с нашей
публикацией в блоге text generation. [here](https://huggingface.co/blog/how-to-generate).

## Named Entity Recognition

Распознавание именованных объектов (NER) - это задача классификации токенов в соответствии с классом, например, идентификация токена
как человека, организации или местоположения. Примером набора данных для распознавания именованных объектов является набор данных CoNLL-2003,
который полностью основан на этой задаче. Если вы хотите точно настроить модель для более сложной задачи, вы можете использовать
[run_ner.py](https://github.com/huggingface/transformers/tree/main/examples/pytorch/token-classification/run_ner.py) script.

Вот пример использования конвейеров для распознавания именованных объектов, в частности, попытки идентифицировать токены как
принадлежащие к одному из 9 классов:

- O, Outside of a named entity
- B-MIS, Beginning of a miscellaneous entity right after another miscellaneous entity
- I-MIS, Miscellaneous entity
- B-PER, Beginning of a person's name right after another person's name
- I-PER, Person's name
- B-ORG, Beginning of an organisation right after another organisation
- I-ORG, Organisation
- B-LOC, Beginning of a location right after another location
- I-LOC, Location

Он использует отлаженную модель на CoNLL-2003,[@stefan-it](https://github.com/stefan-it) from [dbmdz](https://github.com/dbmdz).

In [ ]:
from transformers import pipeline

ner_pipe = pipeline("ner")

sequence = """Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO,
therefore very close to the Manhattan Bridge which is visible from the window."""

В результате будет получен список всех слов, которые были идентифицированы как одна из сущностей из 9 классов, определенных выше.
Вот ожидаемые результаты:

In [ ]:
for entity in ner_pipe(sequence):
    print(entity)

{'entity': 'I-ORG', 'score': 0.9996, 'index': 1, 'word': 'Hu', 'start': 0, 'end': 2}
{'entity': 'I-ORG', 'score': 0.9910, 'index': 2, 'word': '##gging', 'start': 2, 'end': 7}
{'entity': 'I-ORG', 'score': 0.9982, 'index': 3, 'word': 'Face', 'start': 8, 'end': 12}
{'entity': 'I-ORG', 'score': 0.9995, 'index': 4, 'word': 'Inc', 'start': 13, 'end': 16}
{'entity': 'I-LOC', 'score': 0.9994, 'index': 11, 'word': 'New', 'start': 40, 'end': 43}
{'entity': 'I-LOC', 'score': 0.9993, 'index': 12, 'word': 'York', 'start': 44, 'end': 48}
{'entity': 'I-LOC', 'score': 0.9994, 'index': 13, 'word': 'City', 'start': 49, 'end': 53}
{'entity': 'I-LOC', 'score': 0.9863, 'index': 19, 'word': 'D', 'start': 79, 'end': 80}
{'entity': 'I-LOC', 'score': 0.9514, 'index': 20, 'word': '##UM', 'start': 80, 'end': 82}
{'entity': 'I-LOC', 'score': 0.9337, 'index': 21, 'word': '##BO', 'start': 82, 'end': 84}
{'entity': 'I-LOC', 'score': 0.9762, 'index': 28, 'word': 'Manhattan', 'start': 114, 'end': 123}
{'entity': 'I-LO

Обратите внимание, как маркеры последовательности "Обнимающееся лицо" были идентифицированы как организация, а "Нью-Йорк Сити",
"ДАМБО" и "Манхэттенский мост" были идентифицированы как местоположения.

Вот пример распознавания именованных объектов с использованием модели и токенизатора. Процесс заключается в следующем:

1. Создайте экземпляр токенизатора и модели из имени контрольной точки. Модель идентифицируется как модель BERT и загружается
с весами, сохраненными в контрольной точке.
2. Определите последовательность с известными объектами, например, "Обнимающееся лицо" в качестве организации и "Нью-Йорк Сити" в качестве местоположения.
3. Разделите слова на лексемы, чтобы их можно было сопоставить с предсказаниями. Мы используем небольшую хитрость, сначала полностью
кодируя и декодируя последовательность, так что у нас остается строка, содержащая специальные токены.
4. Закодируйте эту последовательность в IDs (специальные токены добавляются автоматически).
5. Извлеките прогнозы, передав входные данные в модель и получив первые выходные данные. В результате
получается распределение по

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = (
    "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, "
    "therefore very close to the Manhattan Bridge."
)

inputs = tokenizer(sequence, return_tensors="pt")
tokens = inputs.tokens()

outputs = model(**inputs).logits
predictions = torch.argmax(outputs, dim=2)

In [ ]:
from transformers import TFAutoModelForTokenClassification, AutoTokenizer
import tensorflow as tf

model = TFAutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = (
    "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, "
    "therefore very close to the Manhattan Bridge."
)

inputs = tokenizer(sequence, return_tensors="tf")
tokens = inputs.tokens()

outputs = model(**inputs)[0]
predictions = tf.argmax(outputs, axis=2)

Это выводит список каждого токена, сопоставленного с соответствующим предсказанием. В отличие от конвейера, здесь каждый
токен имеет предсказание, поскольку мы не удалили "0"-й класс, что означает, что для этого токена не была найдена конкретная сущность
.

В приведенном выше примере "predictions" - это целое число, соответствующее предсказанному классу. Мы можем использовать свойство
"model.config.id2label", чтобы восстановить имя класса, соответствующее номеру класса, который
показан ниже:

In [ ]:
for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

('[CLS]', 'O')
('Hu', 'I-ORG')
('##gging', 'I-ORG')
('Face', 'I-ORG')
('Inc', 'I-ORG')
('.', 'O')
('is', 'O')
('a', 'O')
('company', 'O')
('based', 'O')
('in', 'O')
('New', 'I-LOC')
('York', 'I-LOC')
('City', 'I-LOC')
('.', 'O')
('Its', 'O')
('headquarters', 'O')
('are', 'O')
('in', 'O')
('D', 'I-LOC')
('##UM', 'I-LOC')
('##BO', 'I-LOC')
(',', 'O')
('therefore', 'O')
('very', 'O')
('close', 'O')
('to', 'O')
('the', 'O')
('Manhattan', 'I-LOC')
('Bridge', 'I-LOC')
('.', 'O')
('[SEP]', 'O')

## Summarization

Обобщение - это задача обобщения документа или статьи в виде более короткого текста. Если вы хотите точно настроить
модель для задачи обобщения, вы можете использовать [run_summarization.py](https://github.com/huggingface/transformers/tree/main/examples/pytorch/summarization/run_summarization.py)
script.

Примером набора данных для обобщения является набор данных CNN / Daily Mail, который состоит из длинных новостных статей и был
создан для задачи обобщения. Если вы хотите точно настроить модель для задачи обобщения,
в этом разделе описаны различные подходы. [document](https://github.com/huggingface/transformers/tree/main/examples/pytorch/summarization/README.md).

Вот пример использования конвейеров для обобщения. В нем используется модель Bart, которая была доработана на CNN
/ Набор данных Daily Mail.

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

Поскольку конвейер обобщения зависит от метода `PreTrainedModel.generate()`, мы можем переопределить
аргументы по умолчанию `PreTrainedModel.generate()` непосредственно в конвейере для `max_length` и `min_length`, как показано
ниже. В результате получается следующая сводка:

In [ ]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': ' Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in
the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and
2002 . At one time, she was married to eight men at once, prosecutors say .'}]

Вот пример выполнения обобщения с использованием модели и токенизатора. Процесс заключается в следующем:

1. Создайте токенизатор и модель из имени контрольной точки. Обобщение обычно выполняется с использованием модели кодера-декодера
, такой как "Bart" или "T5".
2. Определите статью, по которой следует обобщить.
3. Добавьте префикс, специфичный для T5, "обобщить:".
4. Используйте метод "PreTrainedModel.generate()" для создания сводки.

В этом примере мы используем модель Google T5. Хотя она была предварительно обучена только для многозадачного смешанного набора данных (включая
CNN / Daily Mail), это дает очень хорошие результаты.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

prosecutors say the marriages were part of an immigration scam. if convicted, barrientos faces two criminal
counts of "offering a false instrument for filing in the first degree" she has been married 10 times, nine of them
between 1999 and 2002.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + ARTICLE, return_tensors="tf", max_length=512)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

prosecutors say the marriages were part of an immigration scam. if convicted, barrientos faces two criminal
counts of "offering a false instrument for filing in the first degree" she has been married 10 times, nine of them
between 1999 and 2002.

## Translation

Письменный перевод - это задача по переводу текста с одного языка на другой. Если вы хотите точно настроить модель для
решения задачи перевода, вы можете использовать [run_translation.py](https://github.com/huggingface/transformers/tree/main/examples/pytorch/translation/run_translation.py) script.

Примером набора данных для перевода является набор данных WMT English to German, который содержит предложения на английском языке в качестве входных
данных и соответствующие предложения на немецком языке в качестве целевых данных. Если вы хотите точно настроить модель для
решения задачи перевода, в этом разделе описаны различные подходы. [document](https://github.com/huggingface/transformers/tree/main/examples/pytorch/translation/README.md).

Вот пример использования конвейеров для выполнения перевода. В нем используется модель T5, которая была предварительно обучена только на
наборе данных с несколькими задачами (включая WMT), но, тем не менее, дает впечатляющие результаты перевода.

In [ ]:
from transformers import pipeline

translator = pipeline("translation_en_to_de")
print(translator("Hugging Face is a technology company based in New York and Paris", max_length=40))

[{'translation_text': 'Hugging Face ist ein Technologieunternehmen mit Sitz in New York und Paris.'}]

Поскольку конвейер трансляции зависит от метода `PreTrainedModel.generate()`, мы можем переопределить
аргументы по умолчанию для `PreTrainedModel.generate()` непосредственно в конвейере, как показано для `max_length` выше.

Вот пример выполнения перевода с использованием модели и токенизатора. Процесс заключается в следующем:

1. Создайте экземпляр токенизатора и модели из имени контрольной точки. Обобщение обычно выполняется с использованием модели кодера-декодера
, такой как "Bart" или "T5".
2. Определите статью, по которой следует обобщить.
3. Добавьте префикс, специфичный для T5, "перевести с английского на немецкий:".
4. Используйте метод `PreTrainedModel.generate()` для выполнения перевода.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

inputs = tokenizer(
    "translate English to German: Hugging Face is a technology company based in New York and Paris",
    return_tensors="pt",
)
outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hugging Face ist ein Technologieunternehmen mit Sitz in New York und Paris.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

inputs = tokenizer(
    "translate English to German: Hugging Face is a technology company based in New York and Paris",
    return_tensors="tf",
)
outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hugging Face ist ein Technologieunternehmen mit Sitz in New York und Paris.

We get the same translation as with the pipeline example.

## Audio classification

Классификация звука присваивает аудиосигналу класс. Набор данных для определения ключевых слов из [SUPERB](https://huggingface.co/datasets/superb) benchmark - это пример набора данных, который можно использовать для точной настройки классификации звука. Этот набор данных содержит десять классов ключевых слов для классификации. Если вы хотите точно настроить модель для классификации звука, ознакомьтесь с [run_audio_classification.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/audio-classification/run_audio_classification.py) script or this [how-to guide](https://huggingface.co/docs/transformers/main/en/./tasks/audio_classification).

Следующие примеры демонстрируют, как использовать [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline) а также модель и токенизатор для вывода классификации звука:

In [ ]:
from transformers import pipeline
from datasets import load_dataset
import torch

torch.manual_seed(42)
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
audio_file = dataset[0]["audio"]["path"]

audio_classifier = pipeline(
    task="audio-classification", model="ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition"
)
predictions = audio_classifier(audio_file)
predictions = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in predictions]
predictions

[{'score': 0.1315, 'label': 'calm'}, {'score': 0.1307, 'label': 'neutral'}, {'score': 0.1274, 'label': 'sad'}, {'score': 0.1261, 'label': 'fearful'}, {'score': 0.1242, 'label': 'happy'}]

Общий процесс использования модели и средства извлечения признаков для классификации звука таков:

1. Создайте средство извлечения признаков и модель из имени контрольной точки.
2. Обработайте аудиосигнал для классификации с помощью средства извлечения признаков.
3. Передайте входные данные через модель и используйте `argmax`, чтобы получить наиболее вероятный класс.
4. Преобразуйте идентификатор класса в имя класса с помощью `id2label`, чтобы получить интерпретируемый результат.

In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

feature_extractor = AutoFeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-ks")
model = AutoModelForAudioClassification.from_pretrained("superb/wav2vec2-base-superb-ks")

inputs = feature_extractor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_ids = torch.argmax(logits, dim=-1).item()
predicted_label = model.config.id2label[predicted_class_ids]
predicted_label

'_unknown_'

## Automatic speech recognition

Автоматическое распознавание речи преобразует аудиосигнал в текст. То [Common Voice](https://huggingface.co/datasets/common_voice) набор данных - это пример набора данных, который можно использовать для точной настройки автоматического распознавания речи. Он содержит аудиофайл говорящего и соответствующее предложение. Если вы хотите точно настроить модель для автоматического распознавания речи, ознакомьтесь с [run_speech_recognition_ctc.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/speech-recognition/run_speech_recognition_ctc.py) or [run_speech_recognition_seq2seq.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/speech-recognition/run_speech_recognition_seq2seq.py) scripts or this [how-to guide](https://huggingface.co/docs/transformers/main/en/./tasks/asr).

Следующие примеры демонстрируют, как использовать [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline), а также модель и токенизатор для автоматического распознавания речи.:

In [ ]:
from transformers import pipeline
from datasets import load_dataset

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
audio_file = dataset[0]["audio"]["path"]

speech_recognizer = pipeline(task="automatic-speech-recognition", model="facebook/wav2vec2-base-960h")
speech_recognizer(audio_file)

{'text': 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL'}

Общий процесс использования модели и процессора для автоматического распознавания речи таков:

1. Создайте экземпляр процессора (который перегруппирует средство извлечения признаков для обработки входных данных и токенизатор для декодирования) и модель по имени контрольной точки.
2. Обработайте аудиосигнал и текст с помощью процессора.
3. Пропустите входные данные через модель и используйте `argmax`, чтобы получить предсказанный текст.
4. Расшифруйте текст с помощью токенизатора, чтобы получить транскрипцию.

In [ ]:
from transformers import AutoProcessor, AutoModelForCTC
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")

inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.batch_decode(predicted_ids)
transcription[0]

'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL'

## Image classification

Подобно классификации текста и звука, классификация изображений присваивает изображению класс. Набор данных [CIFAR-100](https://huggingface.co/datasets/cifar100) является примером набора данных, который можно использовать для точной настройки классификации изображений. Он содержит изображение и соответствующий класс. Если вы хотите точно настроить модель классификации изображений, ознакомьтесь со сценарием [run_image_classification.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/image-classification/run_image_classification.py) или этим [практическим руководством](https://huggingface.co/docs/transformers/main/en/./tasks/image_classification).

Следующие примеры демонстрируют, как использовать [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline), а также модель и токенизатор для вывода классификации изображений:

In [ ]:
from transformers import pipeline

vision_classifier = pipeline(task="image-classification")
result = vision_classifier(
    images="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)
print("\n".join([f"Class {d['label']} with score {round(d['score'], 4)}" for d in result]))

Class lynx, catamount with score 0.4335
Class cougar, puma, catamount, mountain lion, painter, panther, Felis concolor with score 0.0348
Class snow leopard, ounce, Panthera uncia with score 0.0324
Class Egyptian cat with score 0.0239
Class tiger cat with score 0.0229

Общий процесс использования модели и графического процессора для классификации изображений таков:

1. Создайте экземпляр графического процессора и модели из имени контрольной точки.
2. Обработайте изображение для классификации с помощью графического процессора.
3. Передайте входные данные через модель и используйте `argmax`, чтобы получить предсказанный класс.
4. Преобразуйте идентификатор класса в имя класса с помощью `id2label`, чтобы вернуть интерпретируемый результат.

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

feature_extractor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

inputs = feature_extractor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

Egyptian cat